# 문서 로딩 테스트

PDF와 HWP 파일을 로드하고 텍스트를 추출합니다.

In [ ]:
import sys
from pathlib import Path

# 프로젝트 루트 경로
project_root = Path().absolute().parent
sys.path.append(str(project_root))

from src import config
from src.document_loader import DocumentLoader, load_single_document

print(f"데이터 디렉토리: {config.RAW_DATA_DIR}")

## 1. 샘플 문서 로드

In [ ]:
# 이전 노트북에서 저장한 샘플 파일 경로 불러오기
try:
    %store -r sample_pdf
    %store -r sample_hwp
except:
    # 저장된 변수가 없으면 수동으로 선택
    pdf_files = list(config.RAW_DATA_DIR.glob("*.pdf"))
    hwp_files = list(config.RAW_DATA_DIR.glob("*.hwp"))
    
    sample_pdf = pdf_files[0] if pdf_files else None
    sample_hwp = hwp_files[0] if hwp_files else None

print(f"샘플 PDF: {sample_pdf.name if sample_pdf else '없음'}")
print(f"샘플 HWP: {sample_hwp.name if sample_hwp else '없음'}")

## 2. PDF 로딩 테스트

In [ ]:
if sample_pdf:
    print(f"PDF 파일 로딩 중: {sample_pdf.name}")
    
    pdf_doc = load_single_document(str(sample_pdf))
    
    print(f"\n파일명: {pdf_doc['file_name']}")
    print(f"파일 타입: {pdf_doc['file_type']}")
    print(f"추출된 글자 수: {pdf_doc['char_count']:,}")
    print(f"\n텍스트 미리보기 (처음 500자):")
    print("="*60)
    print(pdf_doc['text'][:500])
    print("="*60)
else:
    print("❌ PDF 파일이 없습니다.")

## 3. HWP 로딩 테스트

In [ ]:
if sample_hwp:
    print(f"HWP 파일 로딩 중: {sample_hwp.name}")
    
    hwp_doc = load_single_document(str(sample_hwp))
    
    print(f"\n파일명: {hwp_doc['file_name']}")
    print(f"파일 타입: {hwp_doc['file_type']}")
    print(f"추출된 글자 수: {hwp_doc['char_count']:,}")
    print(f"\n텍스트 미리보기 (처음 500자):")
    print("="*60)
    print(hwp_doc['text'][:500])
    print("="*60)
    
    if hwp_doc['char_count'] == 0:
        print("\n⚠️ HWP 텍스트 추출에 실패했습니다.")
        print("   olefile 방식으로 추출이 안되는 HWP 버전일 수 있습니다.")
        print("   대안: LibreOffice로 PDF 변환 후 사용")
else:
    print("❌ HWP 파일이 없습니다.")

## 4. 전체 문서 로딩 (샘플 10개)

In [ ]:
# 전체 문서 로딩은 시간이 오래 걸리므로 먼저 10개만 테스트
all_files = list(config.RAW_DATA_DIR.glob("*.pdf")) + list(config.RAW_DATA_DIR.glob("*.hwp"))
sample_files = all_files[:10]

print(f"샘플 {len(sample_files)}개 문서 로딩 중...\n")

loaded_docs = []
for file_path in sample_files:
    try:
        doc = load_single_document(str(file_path))
        loaded_docs.append(doc)
        print(f"✅ {doc['file_name']}: {doc['char_count']:,} 글자")
    except Exception as e:
        print(f"❌ {file_path.name}: {e}")

print(f"\n성공: {len(loaded_docs)}개 / 총 {len(sample_files)}개")

## 5. 텍스트 품질 검증

In [ ]:
import numpy as np

if loaded_docs:
    char_counts = [doc['char_count'] for doc in loaded_docs]
    
    print("추출된 텍스트 길이 통계:")
    print(f"  평균: {np.mean(char_counts):,.0f} 글자")
    print(f"  중앙값: {np.median(char_counts):,.0f} 글자")
    print(f"  최소: {np.min(char_counts):,.0f} 글자")
    print(f"  최대: {np.max(char_counts):,.0f} 글자")
    
    # 텍스트가 비어있는 문서 확인
    empty_docs = [doc for doc in loaded_docs if doc['char_count'] == 0]
    if empty_docs:
        print(f"\n⚠️ 텍스트가 추출되지 않은 문서: {len(empty_docs)}개")
        for doc in empty_docs:
            print(f"  - {doc['file_name']} ({doc['file_type']})")
    else:
        print(f"\n✅ 모든 문서에서 텍스트 추출 성공!")

## 6. 문서 로딩 결과 저장

In [ ]:
# 로딩 결과를 다음 노트북에서 사용할 수 있도록 저장
%store loaded_docs

print("✅ 문서 로딩 결과 저장 완료")
print("\n다음 단계: 03_chunking_test.ipynb")

## 7. HWP 추출 실패 시 대안

HWP 파일의 텍스트 추출이 제대로 되지 않는 경우, 아래 대안들을 고려할 수 있습니다:

### 옵션 1: hwp5 라이브러리 사용
```bash
pip install hwp5
```

### 옵션 2: LibreOffice 변환
```bash
# LibreOffice 설치 후
soffice --headless --convert-to pdf:writer_pdf_Export input.hwp --outdir output/
```

### 옵션 3: PDF 파일만 먼저 진행
HWP 파일은 수동으로 PDF 변환 후 사용